In [464]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
from susy_qm import calculate_Hamiltonian

import itertools

In [482]:
potential = 'DW'
cutoff = 4
shots = 2

In [483]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [484]:
operator_pool = []
for i in range(num_qubits):
    #operator_pool.append(qml.Identity(i))
    #operator_pool.append(qml.PauliX(i))
    operator_pool.append(qml.PauliY(i))
    operator_pool.append(qml.PauliZ(i))


In [485]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)

@qml.qnode(dev)
def basis_circuit(basis_state):

    qml.BasisState(basis_state, wires=range(num_qubits))

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [486]:
basis_list = list(itertools.product([0,1], repeat=num_qubits))
basis = []
for _ in range(20):
    energies = []
    for bs in basis_list:
        energies.append(basis_circuit(bs))

    print(basis_list[np.argmin(energies)], energies[np.argmin(energies)])
    basis.append([basis_list[np.argmin(energies)], energies[np.argmin(energies)]])

min_basis, min_energy = min(basis, key=lambda x: x[1])
print("The basis with the lowest energy:", min_basis)
print("Lowest energy:", min_energy)

(1, 0, 0) 0.9065598714743808
(0, 0, 0) 0.9506335329473417
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(0, 0, 0) 0.9506335329473417
(0, 0, 0) 2.3189289829038993
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(1, 0, 0) 0.9065598714743808
(0, 1, 1) 1.6956663473734905
The basis with the lowest energy: (1, 0, 0)
Lowest energy: 0.9065598714743808


In [470]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
basis_state = min_basis

@qml.qnode(dev)
def circuit(times, op_list, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))

    for i, op in enumerate(op_list):
        qml.ApproxTimeEvolution(op, time=times[i], n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [471]:
dev2 = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev2)
def grad_circuit(times, operator_ham, op_list, op_params, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))
    param_index = 0
    for op in op_list:
        qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)
        param_index +=1

    qml.ApproxTimeEvolution(operator_ham, time=times, n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [472]:
def compute_grad(operator, op_list, op_params):
    t = pnp.tensor(0.0, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(t, operator, op_list, op_params)
    return grad

In [473]:
def cost_function(times, op_list):

    times = pnp.tensor(times, requires_grad=True)
    energy = circuit(times, op_list)
    
    return energy

In [474]:
num_steps = 4


#variables
max_iter = 300
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

energies = []

for _ in range(1):

    op_list = []
    op_params = []

    for i in range(num_steps):

        print(f"step: {i}")

        grad_list = []

        for op in operator_pool:
            grad = compute_grad(op, op_list, op_params)
            grad_list.append(abs(grad))

        maxidx = np.argmax(grad_list)
        op_list.append(operator_pool[maxidx])

        # Generate Halton sequence
        num_dimensions = len(op_list)
        num_samples = popsize
        halton_sampler = Halton(d=num_dimensions)
        halton_samples = halton_sampler.random(n=num_samples)
        scaled_samples = 2 * np.pi * halton_samples

        bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]
        
        res = differential_evolution(cost_function,
                                        bounds,
                                        args=(op_list,),
                                        maxiter=max_iter,
                                        tol=tol,
                                        atol=abs_tol,
                                        strategy=strategy,
                                        popsize=popsize,
                                        init=scaled_samples,
                                        )
        
        if i!=0: pre_min_e = min_e
        min_e = res.fun
        pre_op_params = op_params
        op_params = res.x

        print(f"Min E: {min_e}")
        print(res.success)

        if i!=0:
            if abs(pre_min_e - min_e) < 1e-8:
                print("gradient converged")
                op_list.pop()
                op_params = pre_op_params
                break

    energies.append(min_e)


    

step: 0
Min E: 0.8916323801368011
True
step: 1
Min E: 0.8916323801368011
True
gradient converged


In [475]:
min(energies)

np.float64(0.8916323801368011)

In [476]:
op_list

[Y(6)]

In [477]:
dev3 = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev3)
def final_circuit(params):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        pauli_string = qml.pauli.pauli_word_to_string(op)
        qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
        params_index += 1

    return qml.state()

In [478]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0))

0: ─╭|Ψ⟩───────────┤  State
1: ─├|Ψ⟩───────────┤  State
2: ─├|Ψ⟩───────────┤  State
3: ─├|Ψ⟩───────────┤  State
4: ─├|Ψ⟩───────────┤  State
5: ─├|Ψ⟩───────────┤  State
6: ─╰|Ψ⟩──RY(4.24)─┤  State


In [479]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "shots": shots,
        "Optimizer": {
                "name": "differential_evolution",
                "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
                "maxiter": max_iter,
                "tolerance": tol,
                "abs_tolerance": abs_tol,
                "strategy": "randtobest1bin",
                "popsize": popsize,
                "init": "scaled_samples"},
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "min_e_exact": min_eigenvalue.real,
        "min_e": min_e
        }

In [480]:
data

{'potential': 'DW',
 'cutoff': 64,
 'optimizer': 'DE',
 'shots': 2,
 'Optimizer': {'name': 'differential_evolution',
  'bounds': '[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
  'maxiter': 300,
  'tolerance': 0.001,
  'abs_tolerance': 0.001,
  'strategy': 'randtobest1bin',
  'popsize': 20,
  'init': 'scaled_samples'},
 'num steps': 4,
 'basis_state': (0, 0, 0, 0, 0, 0, 0),
 'op_list': ['Y(6)'],
 'min_e_exact': np.float64(0.8916323801370194),
 'min_e': np.float64(0.8916323801368011)}

In [481]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\{}".format(potential)
os.makedirs(base_path, exist_ok=True)
path = os.path.join(base_path,"{}_{}.json".format(potential, cutoff))
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)